https://www.youtube.com/watch?v=m3WwttSBDvo


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

In [ ]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
from milvus import default_server

In [ ]:
default_server.start()

In [ ]:
from langchain.vectorstores import Milvus

In [ ]:
embeddings = OpenAIEmbeddings

In [ ]:
from pymilvus import utility, connections

connections.connect(host='127.0.0.1', port=default_server.listen_port)
utility.drop_collection('LangChainCollection')

In [ ]:
vectordb = Milvus.from_documents(
    {},
    embeddings,
    connections_args = {'host':'127.0.0.1', 'port':default_server.listen_port()}
)

In [ ]:
retriever = Milvus.as_retriever(vectordb, search_kwargs = dict(k = 20))
memory = VectorStoreRetrieverMemory(retriever=retriever)

In [ ]:
about_me = [
    {"input":"My favorite snack is chocolate", 
     "output":"Nice"},
     {"input":"My favorite color is blue", 
     "output":"Cool"},
     {"input":"My favorite sport is futbol", 
     "output":"Sweet"},
     {"input":"my favorite drink is beer", 
     "output":"Good to know"},
]

for example in about_me:
    memory.save_context({"input": example['input']}, {"output":example['output']})

In [ ]:
from langchain_together import Together
llm = Together(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    temperature=0,
    max_tokens=256,
    top_k=1,
    together_api_key=os.getenv("TOGETHER_API_KEY")
)

DEFAULT_TEMPLATE = """The following exchange is a friendly conversation between a human and an AI. The AI is talkative and provides a lot of specific details of its context.
If the AI does not know the answer to the question, it will truthfully sai it does not know

Relevant pieces of previous conversations:
{history}

Current Conversation:
Human: {input}
AI:"""

PROMPT = PromptTemplate(
    input_variables=['history', 'input'], template=DEFAULT_TEMPLATE
)

In [ ]:
conversation_chain = ConversationChain(
    llm=llm,
    prompt=PROMPT,
    memory = memory,
    verbose = True
)

In [ ]:
conversation_chain.predict(input="Hi, my name is Yugi Moto")

In [ ]:
conversation_chain.predict(input="who is my favorite musician?")

In [ ]:
conversation_chain.predict(input='What is my favorite sport')

In [ ]:
conversation_chain.predict(input='What is my name?')

In [ ]:
default_server.stop()
default_server.cleanup()